In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc

In [2]:
sns.set_context('talk')

In [3]:
import warnings
warnings.filterwarnings('ignore', message='Changing the shape of non-C contiguous array')

# Read the data

In [4]:
dfXtrain = pd.read_csv('preprocessed_csv/train_more.csv', index_col='id')
dfXtest = pd.read_csv('preprocessed_csv/test_more.csv', index_col='id')
dfYtrain = pd.read_csv('preprocessed_csv/y_train_more.csv', header=None, names=['INDEX', 'P_TARGET_FLAG'])

In [5]:
x_train = np.array(dfXtrain)
x_test = np.array(dfXtest)

y_train = np.array(dfYtrain['P_TARGET_FLAG'])

# Save routines

In [10]:
dfYtest_stacking = pd.DataFrame({'INDEX': dfXtrain.index, 'P_TARGET_FLAG': np.zeros(x_train.shape[0])})
dfYtest_stacking.head()

,INDEX,P_TARGET_FLAG
0,1,0.0
1,2,0.0
2,4,0.0
3,5,0.0
4,6,0.0


In [11]:
def save_to_file(y, file_name):
    dfYtest_stacking['P_TARGET_FLAG'] = y
    dfYtest_stacking.to_csv('stacking/{}'.format(file_name), index=False)

# Train XGB

In [8]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
XGBC = xgb.XGBClassifier

In [12]:
kwargs = {'objective':'binary:logistic', 'missing': -1, 'seed': 56,
          'n_estimators': 160, 'reg_alpha': 6, 'reg_lambda': 1, 'max_depth': 3, 'colsample_bytree': 1,
          'gamma': 0.008}

clf = XGBC(**kwargs)
clf.fit(x_train, y_train, eval_metric='auc')
proba = clf.predict_proba(x_train)[:, 1]

save_to_file(proba, 'xbgoost_grid.csv')